<a href="https://colab.research.google.com/github/HimeshBhattacharjee/IntelliView/blob/main/IntelliView_AI_Powered_Interview_Coach.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **🤖 IntelliView: AI-Powered Interview Coach**

Welcome to **IntelliView** - your intelligent interview preparation assistant! This interactive notebook contains a complete Streamlit application that helps job seekers:

- 📄 **Analyze resumes** with ATS scoring and detailed feedback
- 🎙️ **Practice interviews** with AI-generated questions
- 📊 **Track progress** with dynamic performance reports
- 💡 **Get coaching** from Gemini AI on your responses

## 🔍 What's Inside?

1. **Smart Resume Analyzer** - Upload your resume and get instant ATS scoring with strengths/weaknesses
2. **AI Interview Simulator** - Practice with tailored questions (General/Technical/HR)
3. **Performance Dashboard** - Visual metrics and downloadable PDF report
4. **Interactive Chat** - Real-time feedback on your interview responses

## 🛠️ How to Use This Notebook

First create Gemini API Key and Ngrok AuthToken, replace it in the code and then follow the steps:
1. Run each cell sequentially (▶️ buttons)
2. The final cell creates `app.py` with the complete application
3. Use `streamlit run app.py` to launch your personal interview coach!

> 💡 **Pro Tip**: The app maintains conversation history and adapts to your resume content!

### **Setting the Stage: Installing Essential Libraries for Our Streamlit Project**

Before diving into building our interactive AI-powered Streamlit application, it's crucial to set up our development environment with all the required tools and libraries. This code cell does exactly that—consider it the **foundation layer** for everything exciting that follows!

🔧 **Here’s what we’re installing and why:**

* **`streamlit`** – The backbone of our project. This lets us build beautiful, interactive web apps straight from Python scripts.
* **`streamlit-option-menu`** – Adds stylish navigation menus to our Streamlit app, enhancing the user experience.
* **`google-generativeai`** – Provides access to cutting-edge generative AI capabilities from Google.
* **`pyngrok`** – Exposes our Streamlit app to the internet securely using ngrok, making local development sharable.
* **`pymupdf`** – A powerful library for working with PDFs (reading, extracting, editing).
* **`fpdf`** – Lightweight PDF generation library; useful for exporting reports or AI-generated outputs.
* **`plotly`** – Enables us to build rich, interactive visualizations directly within Streamlit.
* **`pandas`** – The data wrangling powerhouse, used for handling structured datasets efficiently.
* **`pillow`** – Image processing made simple. Ideal for manipulating images in your Streamlit app.

***Note: Remember to generate an auth token from Ngrok in order to run the streamlit app***

🌐 **Connecting to the World:**
Finally, we authorize **ngrok** using a personal auth token. This allows us to tunnel our local Streamlit server to a publicly accessible web URL—perfect for showcasing your project or collaborating remotely!

> *Think of this step as assembling your superhero toolkit before jumping into action!*

In [ ]:
from google.colab import userdata
import os

NGROK_TOKEN = userdata.get('NGROK_AUTHTOKEN')
os.environ['NGROK_AUTHTOKEN'] = NGROK_TOKEN

In [ ]:
%%capture
!pip install streamlit streamlit-option-menu google-generativeai pyngrok pymupdf fpdf plotly pandas pillow
!ngrok authtoken $NGROK_AUTHTOKEN

---

### **Cell 1: Configuration & Essential Imports – Powering Up the Engine!**

Welcome to the core of our application! In this cell, we're laying the **foundational bricks** for our Streamlit-based AI app by configuring the environment and importing the critical libraries required for the project to function smoothly.

#### 📦 Importing Libraries

We bring in a powerful combination of Python libraries to handle various tasks:

* **`streamlit`** – For building the interactive web app UI.
* **`google.generativeai`** – To tap into Google’s **Gemini 1.5 Flash** model and unlock powerful generative AI capabilities.
* **`fitz (PyMuPDF)`** – Enables reading and extracting content from PDF files effortlessly.
* **`tempfile` & `os`** – For file handling and managing temporary data.
* **`pandas`** – To work seamlessly with data tables and structured datasets.
* **`plotly.express` & `plotly.graph_objects`** – For creating stunning, interactive visualizations.
* **`fpdf`** – To dynamically generate and export PDFs from user inputs or results.
* **`base64` & `json`** – Handle encoded data and configuration structures.
* **`PIL` (Python Imaging Library)** – For handling images and manipulating them as needed.
* **`io`** – For efficient byte-level operations (used in image or file processing).

#### 🔐 API Key Configuration

We securely configure the **Google Generative AI API** using a provided key. This step authenticates our access to Google’s large language model capabilities.

***Note: Remember to create your own Gemini API Key and paste it to the corresponding below code to make it work when you run the Colab notebook.***

#### 🧪 Gemini Model Setup

We initialize the **`gemini-1.5-flash`** model with a custom configuration:

* `temperature`: Controls randomness.
* `top_p` & `top_k`: Influence the diversity of outputs.
* `max_output_tokens`: Sets the maximum size of generated content.

> *This configuration ensures a fine balance between creativity and coherence in AI responses, tailored for both performance and clarity.*

In [ ]:
# Cell 1: Configuration and Imports
%%writefile app.py
import streamlit as st
import google.generativeai as genai
import fitz
import tempfile
import os
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from fpdf import FPDF
import base64
import json
from PIL import Image
import io

GOOGLE_API_KEY = "YOUR_GEMINI_API_KEY_HERE" #REPLACE THIS WITH YOUR OWN GEMINI API KEY TO ENABLE THE WEBSITE
genai.configure(api_key=GOOGLE_API_KEY)
generation_config = {
    "temperature": 0.7,
    "top_p": 1,
    "top_k": 32,
    "max_output_tokens": 4096,
}

model = genai.GenerativeModel(model_name="gemini-1.5-flash", generation_config=generation_config)

---

### **Cell 2: Session State Initialization – Memory Matters!**

To build a smooth, responsive, and interactive app experience, we need our application to **"remember"** things as users interact with it. That’s exactly what this cell does—by initializing values using **`st.session_state`**, we give our Streamlit app a powerful memory!

#### 🧠 Why Session State?

Streamlit reruns the entire script from top to bottom every time a user interacts with the interface. Without session state, we’d lose all intermediate user data—making it impossible to track progress or context across pages. With session state, we maintain continuity, context, and user data throughout the session.

#### 🧩 Here’s what we’re storing:

* **`resume_text`** – Holds the extracted or uploaded resume content.
* **`job_role`** – Stores the selected job role the user is targeting.
* **`chat_history`** – Keeps a record of user-AI interactions (great for a chatbot or assistant-style UI).
* **`current_page`** – Tracks which page of the app the user is currently viewing (like "Landing", "Upload", "Analysis", etc.).
* **`report_data`** – A container to store structured analysis results of the resume.
* **`generated_questions`** – Categorized collections of interview questions generated by the AI:

  * `"general"`: HR-style or open-ended questions.
  * `"technical"`: Role-specific technical questions.
  * `"hr"`: Behavioral or situational HR questions.
* **`active_tab`** – Indicates the currently selected question category tab.
* **`questions_generated`** – Boolean flag to check if questions have already been generated (avoids regenerating on every rerun).
* **`resume_analyzed`** – Flag to confirm whether the resume has already been processed.

> *Think of this step as setting up your app’s brain—one that remembers every detail during the session so your user doesn’t have to start over!*

In [ ]:
# Cell 2: Session State Initialization
%%writefile app.py -a
if "resume_text" not in st.session_state:
    st.session_state.resume_text = ""
if "job_role" not in st.session_state:
    st.session_state.job_role = ""
if "chat_history" not in st.session_state:
    st.session_state.chat_history = []
if "current_page" not in st.session_state:
    st.session_state.current_page = "Landing"
if "report_data" not in st.session_state:
    st.session_state.report_data = {}
if "generated_questions" not in st.session_state:
    st.session_state.generated_questions = { "general": [], "technical": [], "hr": [] }
if "active_tab" not in st.session_state:
    st.session_state.active_tab = "general"
if "questions_generated" not in st.session_state:
    st.session_state.questions_generated = False
if "resume_analyzed" not in st.session_state:
    st.session_state.resume_analyzed = False

---

### **Cell 3: Injecting Custom CSS – Styling with Flair & Function**

Let’s face it—**first impressions matter**, even for apps! In this cell, we supercharge the look and feel of our Streamlit app by injecting **custom CSS styles** that make the interface more polished, modern, and intuitive.

#### 💻 What This Styling Magic Covers:

##### 🌌 Sidebar Glow-Up:

We give the sidebar a sleek **dark gradient background** that adds contrast and professionalism. It also pairs beautifully with our light-text UI.

##### 🏷 App Title & Headers:

* The main title and landing header are given a **vibrant gradient text effect** using `-webkit-background-clip`, ensuring they pop with elegance.
* Aligned center, bold, and large—perfect for making a statement.

##### 🧼 Clean Buttons:

* Custom button styles for the sidebar give users visual feedback on hover and focus.
* Smooth transitions and rounded edges make every interaction feel deliberate and premium.

##### 💬 Chat & Tabs:

* We clean up the chat container with improved spacing and ensure tab lists look neat and minimalistic.

##### 📦 Feature Boxes:

* Feature highlight sections are styled with **soft hover effects, shadows, and border transitions**.
* These UI containers are perfect for highlighting unique functionalities or steps in a workflow.

#### ⚙️ Why Use `unsafe_allow_html=True`?

This enables the raw HTML/CSS code inside the markdown to be rendered properly by Streamlit—without it, our style won’t take effect!

> *With this styling layer, we go beyond functionality and create an experience. It’s not just an app—it’s your brand, beautifully presented.*

In [ ]:
# Cell 3: Custom CSS
%%writefile app.py -a
def inject_custom_css():
    st.markdown("""
    <style>
        /* Sidebar styling */
        [data-testid="stSidebar"] {
            background: linear-gradient(to bottom, #1c1c1c, #383838) !important;
        }

        /* App title styling */
        .app-title {
            font-size: 32px !important;
            font-weight: 700 !important;
            color: #ffffff !important;
            margin-bottom: 20px !important;
            text-align: center;
            background: linear-gradient(90deg, #4e54c8, #8f94fb);
            -webkit-background-clip: text;
            -webkit-text-fill-color: transparent;
        }

        /* Landing page styling */
        .landing-header {
            font-size: 42px !important;
            font-weight: 700 !important;
            background: linear-gradient(90deg, #4e54c8, #8f94fb);
            -webkit-background-clip: text;
            -webkit-text-fill-color: transparent;
            text-align: center;
            margin-bottom: 5px !important;
        }

        /* Delete button styling */
        .delete-btn {
            font-size: 4px !important;
            padding: 2px 6px !important;
            margin-left: 8px !important;
        }

        /* Chat container */
        .chat-container {
            margin-bottom: 2px;
            overflow-y: auto;
        }

        /* Tab styling */
        .stTabs [role="tablist"] {
            margin-bottom: 0px;
        }

        /* Sidebar button styling */
        .stSidebar .stButton button {
            width: 100%;
            margin: 5px 0;
            text-align: center;
            border-radius: 5px;
            transition: all 0.3s ease;
            background-color: rgba(255, 255, 255, 0.05);
            color: white;
            border: none;
        }

        .stSidebar .stButton button:hover {
            background-color: rgba(255, 255, 255, 0.4);
        }

        .stSidebar .stButton button:focus:not(:active) {
            background-color: #4e54c8;
            font-weight: bold;
            color: white;
        }

        /* Feature box styling */
        .feature-box {
            padding: 15px;
            border-radius: 10px;
            background: rgba(78, 84, 200, 0.1);
            height: 100%;
            display: flex;
            gap: 4px;
            flex-direction: column;
            justify-content: space-between;
            transition: all 0.3s ease;
            box-shadow: 0 2px 4px rgba(0,0,0,0.05);
            border: 3px solid rgba(78, 84, 200, 0.2);
            margin: 10px 0;
        }

        .feature-box:hover {
            cursor: pointer;
            background: rgba(78, 84, 200, 0.2) !important;
            transform: translateY(-3px);
            box-shadow: 0 4px 8px rgba(0,0,0,0.1);
            border-color: rgba(78, 84, 200, 0.6);
        }
    </style>
    """, unsafe_allow_html=True)

---

### **Cell 4: Utility Functions – Resume Parsing & Gemini-Powered Analysis**

This cell introduces the **first wave of core utility functions**—the engine room of our resume analyzer app. These methods are essential for transforming raw input into meaningful insights powered by Gemini 1.5 Flash.

---

#### 📄 `extract_text_from_pdf(uploaded_file)`

This function handles the **PDF text extraction** process using the `PyMuPDF (fitz)` library:

* Temporarily stores the uploaded resume file using Python’s `tempfile`.
* Iterates over each page to extract all textual content.
* Cleans up temporary files post-extraction to avoid clutter.

> **Why it matters**: This ensures we work only with raw text, making downstream AI analysis smoother and error-free.

---

#### 🤖 `analyze_resume_with_gemini(resume_text, job_role="")`

This function is where the **AI magic happens**:

* Uses the Google Gemini 1.5 Flash model to deeply **analyze resume content**.
* Dynamically includes an optional job role to **tailor the feedback** and simulate ATS (Applicant Tracking System) evaluation.
* Returns structured output in a consistent, easy-to-parse format:

  * 🔢 **ATS Score** (0–100)
  * ✅ Strengths
  * ⚠️ Weaknesses
  * 💡 Recommendations

> **Gemini's Role:** Acts as a smart evaluator giving honest feedback with pinpoint suggestions, mimicking the criteria recruiters and ATS systems use.

---

**These functions lay the groundwork for an intelligent resume assistant—one that reads, analyzes, and offers real-time guidance to help users land their dream job!**

In [ ]:
# Cell 4: Utility Functions (Part 1)
%%writefile app.py -a
def extract_text_from_pdf(uploaded_file):
    with tempfile.NamedTemporaryFile(delete=False, suffix=".pdf") as tmp:
        tmp.write(uploaded_file.getvalue())
        tmp_path = tmp.name
    doc = fitz.open(tmp_path)
    text = ""
    for page in doc:
        text += page.get_text()
    os.unlink(tmp_path)
    return text

def analyze_resume_with_gemini(resume_text, job_role=""):
    prompt = f"""
    Analyze this resume for ATS scoring and provide detailed feedback:

    RESUME CONTENT:
    {resume_text}

    {f"TARGET JOB ROLE: {job_role}" if job_role else ""}

    Provide output in this EXACT structure:

    ATS Score: (0-100)

    Strengths:
    - Bullet point 1
    - Bullet point 2

    Weaknesses:
    - Bullet point 1
    - Bullet point 2

    Recommendations:
    - Bullet point 1
    - Bullet point 2
    """

    try:
        response = model.generate_content(prompt)
        return response.text
    except Exception as e:
        st.error(f"An error occurred: {e}")
        return ""

---

### **Cell 5: Utility Functions – Question Generation & AI Interview Coaching**

This cell introduces the **second batch of utility functions**, focused on crafting an immersive **mock interview experience** using your resume. From generating targeted questions to enabling natural conversation with an AI coach — this is where interview preparation meets intelligence.

---

#### 🎯 `generate_all_questions()`

This function is a **powerhouse for question generation**. Here's what it does:

* Iterates through three key interview domains:
  ✅ **General**
  💻 **Technical**
  🧍‍♂️ **HR (Human Resources)**
* For each category, it sends a structured prompt to Gemini 1.5 Flash using your resume's content (limited to the first 2000 characters for relevance).
* Parses the AI response into a clean, numbered list and stores it in the session state.

> **Why it matters**: This auto-generates realistic interview questions that reflect your resume and target role, allowing you to prep smart and focus on the right areas.

---

#### 💬 `generate_chat_response(prompt, category)`

This function brings your personal **AI Interview Coach – *IntelliView*** to life:

* Maintains context with the latest chat history.
* Reads your resume and understands the category (general, technical, HR).
* Provides **dynamic feedback**, tips, and suggestions on your responses in a human-like tone.

> **Gemini’s role**: Powers intelligent, interactive responses that help sharpen your communication, clarity, and confidence.

---
**With this layer of automation, you're not just preparing for interviews—you’re simulating them with an AI that thinks like a recruiter.**

In [ ]:
# Cell 5: Utility Functions (Part 2)
%%writefile app.py -a
def generate_all_questions():
    """Generate all question categories at once"""
    categories = ["general", "technical", "hr"]
    for category in categories:
        prompt = f"""
        Generate 5 {category} interview questions based on this resume:
        {st.session_state.resume_text[:2000]}

        Return the questions as a numbered list. And don't add any introductory sentence at all - just straight to the questions.
        """
        try:
            response = model.generate_content(prompt)
            questions = [q.strip() for q in response.text.split("\n") if q.strip()]
            st.session_state.generated_questions[category] = questions
        except Exception as e:
            st.error(f"Error generating {category} questions: {e}")
            st.session_state.generated_questions[category] = []
    st.session_state.questions_generated = True

def generate_chat_response(prompt, category):
    context = f"""
    You are an AI Interview Coach named IntelliView.
    The user is preparing for {category} interviews with this resume context:
    {st.session_state.resume_text[:2000]}

    Current conversation:
    {"".join([f"{'User' if i%2 else 'AI'}: {msg}" for i, msg in enumerate(st.session_state.chat_history[-4:])])}

    New input: {prompt}

    Provide helpful coaching advice and feedback on the user's response.
    """

    try:
        response = model.generate_content(context)
        return response.text
    except Exception as e:
        st.error(f"An error occurred: {e}")
        return ""

---

### **Cell 6: Utility Functions – Dynamic Reporting & Visualization**

This final utility batch transforms your interview preparation into actionable insights and stunning visuals. It does three things:

1. 📋 Evaluates your mock interview performance.
2. 📄 Generates a professional PDF report.
3. 🥧 Visualizes your strengths using a pie chart.

---

#### 📈 `generate_dynamic_report()`

This function calls Gemini to create a **deep-dive JSON report** that evaluates:

* ✅ **Overall Readiness Score**
* 📉 Strengths & Weaknesses
* 📋 Personalized Recommendations
* 🧠 Textual Performance Analysis
* 📊 Skill Metrics with trends (like Communication, Problem Solving, etc.)

> *It uses both your resume and recent chat history to deliver a contextual report — just like a real interview coach would.*

---

#### 🧾 `create_pdf_report()`

This function converts the JSON report into a **PDF file** that you can download or share. It includes:

* Title & Score Summary
* Metrics with Delta Improvements
* Structured Sections: Strengths, Weaknesses, Recommendations, Analysis

> *Perfect for keeping a record of your interview readiness or presenting it to a mentor.*

---

#### 🥧 `create_pie_chart()`

This function builds a clean, elegant **donut-style pie chart** using Plotly:

* Visualizes your metric-wise strengths.
* Colors and layout are optimized for readability and clarity.

> *A visual touch to help you quickly assess areas of excellence and improvement.*

---

**With this cell, your resume review app becomes a full-fledged AI interview readiness platform — from analysis to reporting and visualization.**

In [ ]:
# Cell 6: Utility Functions (Part 3)
%%writefile app.py -a
def generate_dynamic_report():
    with st.spinner("Generating dynamic report..."):
        prompt = f"""
        Create a comprehensive dynamic interview readiness report for this candidate:

        Resume Context:
        {st.session_state.resume_text[:3000]}

        Chat History:
        {"".join(st.session_state.chat_history[-6:])}

        Provide a JSON object with:
        1. overall_score (0-100)
        2. metrics (list of dicts with name, score, delta)
        3. strengths (list)
        4. weaknesses (list)
        5. recommendations (list)
        6. analysis (text)

        Example format:
        {{
            "overall_score": 85,
            "metrics": [
                {{"name": "Technical Knowledge", "score": 90, "delta": "+5"}},
                {{"name": "Communication", "score": 80, "delta": "+10"}},
                {{"name": "Problem Solving", "score": 85, "delta": "+15"}}
            ],
            "strengths": ["Strong technical background", "Clear communication"],
            "weaknesses": ["Could improve time management", "Needs more examples"],
            "recommendations": ["Practice time management", "Prepare more examples"],
            "analysis": "Overall strong candidate with good technical skills..."
        }}
        """

        try:
            response = model.generate_content(prompt)
            report_json = json.loads(response.text.replace('```json', '').replace('```', '').strip())
            st.session_state.report_data = report_json
        except Exception as e:
            st.error(f"Error generating report: {str(e)}")
            st.session_state.report_data = {
                "overall_score": 0,
                "metrics": [],
                "strengths": [],
                "weaknesses": [],
                "recommendations": [],
                "analysis": "Could not generate report"
            }

def create_pdf_report():
    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial", size=12)
    pdf.set_font("Arial", 'B', 16)
    pdf.cell(200, 10, txt="IntelliView Interview Readiness Report", ln=1, align='C')
    pdf.ln(10)
    pdf.set_font("Arial", 'B', 14)
    pdf.cell(200, 10, txt="Overall Assessment", ln=1)
    pdf.set_font("Arial", size=12)
    pdf.multi_cell(0, 10, txt=f"Overall Score: {st.session_state.report_data.get('overall_score', 'N/A')}/100")
    pdf.ln(5)
    pdf.set_font("Arial", 'B', 14)
    pdf.cell(200, 10, txt="Key Metrics", ln=1)
    pdf.set_font("Arial", size=12)
    for metric in st.session_state.report_data.get('metrics', []):
        pdf.multi_cell(0, 10, txt=f"{metric['name']}: {metric['score']} ({metric.get('delta', '')})")
    sections = ['strengths', 'weaknesses', 'recommendations', 'analysis']
    titles = ['Strengths', 'Areas for Improvement', 'Recommendations', 'Performance Analysis']
    for section, title in zip(sections, titles):
        pdf.ln(5)
        pdf.set_font("Arial", 'B', 14)
        pdf.cell(200, 10, txt=title, ln=1)
        pdf.set_font("Arial", size=12)
        content = st.session_state.report_data.get(section, [])
        if isinstance(content, list):
            content = "\n".join([f"- {item}" for item in content])
        pdf.multi_cell(0, 10, txt=content)
    temp_file = tempfile.NamedTemporaryFile(delete=False, suffix=".pdf")
    pdf.output(temp_file.name)
    return temp_file.name

def create_pie_chart():
    if not st.session_state.report_data or not st.session_state.report_data.get('metrics'):
        return None
    metrics = st.session_state.report_data['metrics']
    df = pd.DataFrame(metrics)
    fig = go.Figure(data=[go.Pie(
        labels=df['name'],
        values=df['score'],
        hole=0.3,
        marker_colors=px.colors.qualitative.Plotly,
        textinfo='label+percent',
        insidetextorientation='radial'
    )])
    fig.update_layout(
        title="Skill Distribution",
        height=400,
        margin=dict(l=20, r=20, t=40, b=20),
        showlegend=False
    )
    return fig

---

### **Cell 7: UI Section – Landing Page**

This function defines the **initial landing interface** for the IntelliView app. It serves as the welcome screen and showcases the core value proposition of the platform.

---

#### 🎯 `landing_page()`

* Creates a **visually centered headline**:

  > *“IntelliView: Your AI Interview Coach”*

* Adds a subheading with a concise value statement:

  > “Your complete solution for interview preparation and career advancement”

* Displays **three feature highlights** using emoji icons and brief descriptions:

  * 📄 **Smart Resume Analysis** – ATS-based scoring & feedback.
  * 🎙️ **AI Interview Practice** – Role-specific, interactive Q\&A coaching.
  * 📊 **Performance Analytics** – In-depth skill metrics and progress tracking.

> *Each feature is laid out in its own column using Streamlit's layout API to ensure a clean, grid-style design.*

---

**This landing section gives users a quick overview of what IntelliView offers and invites them to dive deeper into the app’s functionalities.**

In [ ]:
# Cell 7: UI Sections - Landing Page
%%writefile app.py -a
def landing_page():
    st.markdown('<div class="landing-header">IntelliView: Your AI Interview Coach</div>', unsafe_allow_html=True)
    st.markdown("""
    <div style="text-align: center; font-size: 18px; margin: 30px 0;">
    Your complete solution for interview preparation and career advancement
    </div>
    """, unsafe_allow_html=True)
    features = [
        ("📄", "Smart Resume Analysis", "Get detailed feedback on your resume with ATS scoring"),
        ("🎙️", "AI Interview Practice", "Practice with realistic interview questions and feedback"),
        ("📊", "Performance Analytics", "Track your progress with detailed metrics and reports")
    ]
    cols = st.columns(3)
    for i, (icon, title, desc) in enumerate(features):
        with cols[i]:
            st.markdown(f"""
            <div class="feature-box">
                <div>
                    <div style="font-size: 24px; margin-bottom: 10px; text-align: center;">{icon}</div>
                    <p style="font-size: 28px; font-weight: bold; text-align: center;">{title}</p>
                    <p style="text-align: center;">{desc}</p>
                </div>
            </div>
            """, unsafe_allow_html=True)

---

### **Cell 8: UI Section – Resume Analyzer (with PDF Visualization)**

This function builds an **interactive UI** in Streamlit where users can upload their resume and get a detailed **ATS-based evaluation** powered by the Gemini model.

---

#### 🔍 `resume_analyzer()`

- **User Inputs:**
  - 📎 Upload a resume in **PDF format**.
  - 🎯 Optionally specify the **target job role** for personalized analysis.

- **Preview Features:**
  - 👁️ Resume is displayed inline using **base64-encoded iframe rendering**.
  - 📜 Full resume text is extracted using **PyMuPDF** for further processing.

- **Trigger:**
  - Clicking the **“Analyze”** button:
    - Extracts and processes the uploaded resume text.
    - Calls `analyze_resume_with_gemini()` for intelligent ATS feedback.
    - Generates mock interview questions and a report.
    - Reloads the app to display the analysis.

- **Displayed Results:**
  - 📊 **ATS Score** shown via **Plotly bar chart** and `st.metric`.
  - ✅ **Strengths**, ⚠️ **Weaknesses**, and 📝 **Recommendations** parsed from Gemini's output and displayed in a clean layout using columns.

---

**This section enables users to visualize their resume, receive AI-driven feedback, and take actionable steps to improve their job-readiness.**

In [ ]:
# Cell 8: UI Sections - Resume Analyzer (with PDF visualization)
%%writefile app.py -a
def resume_analyzer():
    st.markdown(
        """
        <h1 style="
            font-size: 3em;
            font-weight: 800;
            background: linear-gradient(to top, #4e54c8, #ffffff);
            -webkit-background-clip: text;
            -webkit-text-fill-color: transparent;
            ">
            Resume Analyzer
        </h1>
        """,
        unsafe_allow_html=True)
    st.markdown("Upload your resume for ATS optimization and personalized feedback")
    uploaded_file = st.file_uploader("Choose PDF", type=["pdf"])
    if uploaded_file is not None:
        with tempfile.NamedTemporaryFile(delete=False, suffix=".pdf") as tmp:
            tmp.write(uploaded_file.getvalue())
            tmp_path = tmp.name
        st.subheader("Resume Preview", anchor=False)
        with open(tmp_path, "rb") as f:
            base64_pdf = base64.b64encode(f.read()).decode('utf-8')
            pdf_display = f'<iframe src="data:application/pdf;base64,{base64_pdf}" width="700" height="1000" type="application/pdf" style="border: 1px solid #4e54c8; border-radius: 5px;"></iframe>'
            st.markdown(pdf_display, unsafe_allow_html=True)
        doc = fitz.open(tmp_path)
        st.session_state.resume_text = ""
        for page in doc:
            st.session_state.resume_text += page.get_text()
        doc.close()
        os.unlink(tmp_path)
    st.session_state.job_role = st.text_input("Target Job Role (Optional)")
    if st.button("Analyze"):
        if uploaded_file:
            with st.spinner("Analyzing your resume..."):
                st.session_state.analysis = analyze_resume_with_gemini(st.session_state.resume_text, st.session_state.job_role)
                generate_all_questions()
                generate_dynamic_report()
                st.session_state.resume_analyzed = True
                st.rerun()
    if st.session_state.get("resume_analyzed") and st.session_state.get("resume_text"):
        analysis = st.session_state.get("analysis", "")
        st.subheader("ATS Score & Feedback", anchor=False)
        if "ATS Score:" in analysis:
            score = int(analysis.split("ATS Score:")[1].split("/100\n")[0].strip())
            st.metric("Your ATS Score", f"{score}/100")
            fig = px.bar(
                x=["ATS Score"],
                y=[score],
                range_y=[0, 100],
                text=[score],
                color_discrete_sequence=["green" if score > 70 else "orange" if score > 50 else "red"]
            )
            fig.update_layout(yaxis_title="Score", xaxis_title="", showlegend=False)
            st.plotly_chart(fig, use_container_width=True)
        col1, col2 = st.columns(2)
        if "Strengths:" in analysis:
            with col1:
                st.subheader("✅ Strengths", anchor=False)
                strengths = analysis.split("Strengths:")[1].split("Weaknesses:")[0].strip()
                st.markdown(strengths)
        if "Weaknesses:" in analysis:
            with col2:
                st.subheader("⚠️ Weaknesses", anchor=False)
                weaknesses = analysis.split("Weaknesses:")[1].split("Recommendations:")[0].strip()
                st.markdown(weaknesses)
        if "Recommendations:" in analysis:
            st.subheader("📝 Recommendations", anchor=False)
            recs = analysis.split("Recommendations:")[1].strip()
            st.markdown(recs)

---

### **Cell 9: UI Section – AI Interview Coach**

This function implements the **AI Interview Coach**, a feature that helps users prepare for job interviews by offering **general**, **technical**, and **HR/situational** interview questions. The coach allows the user to practice responses to these questions, simulating a real-life interview environment.

---

#### 🧑‍🏫 **How it Works**:
The **AI Interview Coach** is divided into three tabs, each dedicated to a different type of interview question:

1. **General Interview Questions**:
   - Displays a list of common interview questions, which are relevant to almost any job role.
   - These questions are dynamically generated based on the user’s resume analysis.

2. **Technical Interview Questions**:
   - Offers questions based on the technical skills and experience from the resume.
   - This section tests the user's knowledge and problem-solving abilities in their field.

3. **HR/Situational Interview Questions**:
   - Provides questions that focus on behavioral and situational assessments, such as how the candidate handles stress, teamwork, and ethical dilemmas.

Each tab provides a list of **suggested questions** to guide the user’s preparation. If no questions are generated, a friendly reminder appears asking the user to first analyze their resume.

---

#### 💬 **Interactive Chat**:
At the bottom of each tab, users can interact with the AI **through a chat interface**. The conversation is designed to simulate a real interview, allowing the user to respond to the questions:

- **Chat History**: Displays the history of the conversation with the AI, providing an ongoing dialogue for better engagement.
- **Message Deletion**: Users can delete individual messages from the chat by clicking on the trash icon, making the experience customizable and neat.

---

#### 💡 **Key Features**:
- **Tabs for Interview Questions**: Seamlessly toggle between general, technical, and HR questions.
- **Dynamic Question Generation**: Questions are automatically generated based on the user’s resume after the analysis is completed.
- **Real-time Chat**: Allows users to respond to questions, receive feedback, and refine answers.
- **Message Management**: Users can delete specific chat messages, giving them control over the conversation history.

---

#### 🚀 **How to Use**:
1. **Analyze Your Resume**: Start by uploading and analyzing your resume to generate customized questions.
2. **Select a Tab**: Choose between General, Technical, or HR questions to focus on.
3. **Respond to Questions**: Type your responses in the chat interface for personalized feedback.
4. **Delete Messages**: Clean up the conversation by deleting messages you no longer want to keep.

---

This feature provides a **hands-on** and **interactive** way to prepare for job interviews, with detailed feedback that simulates a real interview experience. It offers a **user-friendly** interface that encourages active participation, improving interview readiness over time.

**Note**: Ensure your resume is analyzed before using this section to get personalized questions and responses.

In [ ]:
# Cell 9: UI Sections - Interview Coach
%%writefile app.py -a
def interview_coach():
    st.markdown(
        """
        <h1 style="
            font-size: 3em;
            font-weight: 800;
            background: linear-gradient(to top, #4e54c8, #ffffff);
            -webkit-background-clip: text;
            -webkit-text-fill-color: transparent;
            ">
            AI Interview Coach
        </h1>
        """,
        unsafe_allow_html=True)
    tab1, tab2, tab3 = st.tabs(["General Questions", "Technical Questions", "HR/Situational Questions"])
    with tab1:
        st.subheader("General Interview Questions", anchor=False)
        questions = st.session_state.generated_questions["general"]
        if questions:
            st.markdown("#### Suggested Questions:")
            for q in questions:
                st.markdown(f"{q}")
        else:
            st.info("No general questions generated yet. Please analyze your resume first.")
        st.markdown("---")
    with tab2:
        st.subheader("Technical Interview Questions", anchor=False)
        questions = st.session_state.generated_questions["technical"]
        if questions:
            st.markdown("#### Suggested Questions:")
            for q in questions:
                st.markdown(f"{q}")
        else:
            st.info("No technical questions generated yet. Please analyze your resume first.")
        st.markdown("---")
    with tab3:
        st.subheader("HR/Situational Questions", anchor=False)
        questions = st.session_state.generated_questions["hr"]
        if questions:
            st.markdown("#### Suggested Questions:")
            for q in questions:
                st.markdown(f"{q}")
        else:
            st.info("No HR questions generated yet. Please analyze your resume first.")
        st.markdown("---")
    st.markdown('<div class="chat-container">', unsafe_allow_html=True)
    for i, msg in enumerate(st.session_state.chat_history):
        cols = st.columns([10, 1])
        with cols[0]:
            if i % 2 == 0:
                with st.chat_message("user"):
                    st.markdown(msg)
            else:
                with st.chat_message("assistant"):
                    st.markdown(msg)
        with cols[1]:
            if st.button("🗑️", key=f"del_{i}", help="Delete this message"):
                st.session_state.chat_history.pop(i)
                st.rerun()
    st.markdown('</div>', unsafe_allow_html=True)
    if prompt := st.chat_input("Type your response..."):
        st.session_state.chat_history.append(prompt)
        with st.spinner("Analyzing your response..."):
            response = generate_chat_response(prompt, st.session_state.active_tab)
            st.session_state.chat_history.append(response)
        st.rerun()

---

### **Cell 10: UI Section – Interview Readiness Report**

This function implements the **Interview Readiness Report** dashboard, which provides a detailed analysis of the user’s performance after analyzing their resume. The report includes an **overall score**, key performance metrics, strengths, weaknesses, and recommendations, along with the ability to download the report as a PDF.

---

#### 🧑‍🏫 **How it Works**:
The **Interview Readiness Report** displays a comprehensive breakdown of the user’s interview preparedness based on their resume analysis. It is divided into several sections, each offering insights into various aspects of the user’s qualifications and performance:

1. **Overall Score**:
   - Displays the overall readiness score out of 100, giving the user a clear understanding of their preparedness.

2. **Key Metrics**:
   - Provides detailed metrics based on specific areas of evaluation, such as skill proficiency, experience alignment, and resume optimization.
   - A pie chart is used to visually represent the distribution of scores across different metrics.

3. **Strengths & Weaknesses**:
   - Highlights the user’s strengths and areas for improvement.
   - The strengths section focuses on key skills or experiences that stand out positively in the resume.
   - The weaknesses section pinpoints areas that could be enhanced for better interview readiness.

4. **Recommendations**:
   - Suggests actions or improvements that the user can undertake to increase their chances of success during an interview.

5. **Performance Analysis**:
   - Provides an in-depth analysis of the user’s overall performance, summarizing critical feedback for each aspect of the resume.

---

#### 📊 **Key Features**:
- **Dynamic Report**: The report is generated based on the analysis of the user’s resume, making it highly personalized.
- **Visual Representation**: A pie chart gives a quick, visual snapshot of the user’s overall performance.
- **Expandable Sections**: Key sections like Strengths, Weaknesses, and Recommendations can be expanded to explore further details.
- **PDF Download**: Users can generate a downloadable PDF report for offline review or sharing.

---

#### 🚀 **How to Use**:
1. **Analyze Your Resume**: Ensure your resume is uploaded and analyzed before accessing the report.
2. **View Key Metrics**: Review the overall score and key metrics to understand your strengths and areas of improvement.
3. **Explore Recommendations**: Expand the sections to read detailed feedback on your performance and areas for improvement.
4. **Download the Report**: Click the “Generate PDF Report” button to create a downloadable PDF of your interview readiness report.

---

#### 📥 **Download Your Report**:
Once the report is generated, you can download it as a **PDF** for your records. Simply click the **"Generate PDF Report"** button to download the document. This report will serve as a valuable reference for further enhancing your interview preparation.

---

This feature provides users with **personalized feedback** on their interview readiness, highlighting both **strong points** and **areas for improvement**, and offering actionable **recommendations**. It empowers users to **track progress** and refine their resumes or responses based on detailed performance metrics.

In [ ]:
# Cell 10: UI Sections - Report Dashboard
%%writefile app.py -a
def report_dashboard():
    st.markdown(
        """
        <h1 style="
            font-size: 3em;
            font-weight: 800;
            background: linear-gradient(to top, #4e54c8, #ffffff);
            -webkit-background-clip: text;
            -webkit-text-fill-color: transparent;
            ">
            Interview Readiness Report
        </h1>
        """,
        unsafe_allow_html=True)
    if not st.session_state.report_data:
        st.info("Please analyze your resume first to generate a report")
        return
    st.subheader("Overall Assessment", anchor=False)
    col1, col2, col3 = st.columns(3)
    with col1:
        st.metric("Overall Score",
                 f"{st.session_state.report_data.get('overall_score', 'N/A')}/100")
    st.subheader("Key Metrics", anchor=False)
    metrics = st.session_state.report_data.get('metrics', [])
    pie_chart = create_pie_chart()
    if pie_chart:
        st.plotly_chart(pie_chart, use_container_width=True)
    cols = st.columns(len(metrics))
    for i, metric in enumerate(metrics):
        with cols[i]:
            st.metric(metric['name'],
                     metric['score'],
                     metric.get('delta', ''))
    sections = [
        ("✅ Strengths", "strengths"),
        ("⚠️ Weaknesses", "weaknesses"),
        ("📝 Recommendations", "recommendations"),
        ("🔍 Performance Analysis", "analysis")
    ]
    for title, key in sections:
        with st.expander(title, expanded=True):
            items = st.session_state.report_data.get(key, [])
            if isinstance(items, list):
                for item in items:
                    st.markdown(f"- {item}")
            else:
                st.markdown(items)
    st.subheader("Download Report", anchor=False)
    if st.button("Generate PDF Report"):
        pdf_path = create_pdf_report()
        with open(pdf_path, "rb") as f:
            pdf_bytes = f.read()
        os.unlink(pdf_path)
        b64 = base64.b64encode(pdf_bytes).decode()
        href = f'<a href="data:application/pdf;base64,{b64}" download="IntelliView_Report.pdf">Download PDF Report</a>'
        st.markdown(href, unsafe_allow_html=True)

---

### **Cell 11: UI Section – Navigation Sidebar**

This function creates a **Navigation Sidebar** for the app, allowing users to seamlessly navigate between different sections such as the **Home**, **Resume Analyzer**, **Interview Coach**, and **Report**. The sidebar enhances user experience by providing quick access to various pages within the application.

---

#### 🧩 **How it Works**:
The navigation sidebar is built using **Streamlit**'s sidebar component and is designed to facilitate smooth transitions between different sections of the application. Here’s how it functions:

1. **Custom CSS Injection**:
   - The `inject_custom_css()` function is used to apply custom styling to the sidebar, ensuring it aligns with the overall design of the app.

2. **App Title**:
   - The title **"IntelliView"** is displayed at the top of the sidebar, giving users a clear indication of the app they are using.

3. **Page Buttons**:
   - The sidebar contains buttons that represent different pages in the app. Each button is labeled with the name of the section (e.g., "Home", "Resume Analyzer", "Interview Coach", "Report").
   - When a button is clicked, the corresponding page is loaded by updating the `st.session_state.current_page` and triggering a rerun of the app to display the selected page.

4. **Navigation Flow**:
   - The user can easily navigate through the app by clicking any of the buttons, which directly correspond to the following sections:
     - **Home**: The landing page of the app.
     - **Resume Analyzer**: Where users can upload and analyze their resumes.
     - **Interview Coach**: Provides personalized interview questions and coaching.
     - **Report**: Displays the interview readiness report based on the resume analysis.

---

#### 🎨 **Key Features**:
- **Seamless Navigation**: The sidebar ensures users can easily move between pages without losing context or disrupting their flow.
- **Custom Styling**: The sidebar’s appearance is enhanced with custom CSS, providing a more cohesive look that matches the app’s design.
- **Responsive Buttons**: Buttons are designed to expand to the container width, making them easy to interact with.

---

#### 🚀 **How to Use**:
1. **Click a Page Button**: Simply click on one of the page buttons in the sidebar (e.g., "Home", "Resume Analyzer", "Interview Coach", "Report") to navigate to the corresponding section.
2. **Stay in Context**: The sidebar ensures that the current page is always easily accessible from any part of the app, making it simple to switch between sections without hassle.
3. **Custom Styling**: Enjoy a polished user interface with the injected custom CSS, which ensures the sidebar aligns perfectly with the overall app design.

---

The **Navigation Sidebar** enhances the user experience by providing a quick and intuitive way to navigate between the various sections of the **IntelliView** application. It ensures users can access all the key features, such as **analyzing resumes**, **coaching for interviews**, and reviewing the **interview readiness report**, with just a few clicks.

In [ ]:
# Cell 11: Navigation Sidebar
%%writefile app.py -a
def navigation_sidebar():
    inject_custom_css()
    st.sidebar.markdown('<div class="app-title">IntelliView</div>', unsafe_allow_html=True)
    pages = {
        "Home": "Landing",
        "Resume Analyzer": "Resume Analyzer",
        "Interview Coach": "Interview Coach",
        "Report": "Report"
    }
    for label, page in pages.items():
        if st.sidebar.button(label, key=f"nav_{page}", use_container_width=True):
            st.session_state.current_page = page
            st.rerun()

---

### **Cell 12: Main App Flow**

This function, `main()`, defines the core logic of the **IntelliView** application. It is responsible for setting up the page configuration, handling the page routing, and managing the navigation flow within the app.

---

#### 🧩 **How it Works**:
The `main()` function orchestrates the setup and navigation of the app. Here's how the function flows:

1. **Page Configuration**:
   - **`st.set_page_config()`** is used to configure the initial settings for the Streamlit app. The configuration parameters are:
     - **`page_title`**: Sets the title of the page to "IntelliView".
     - **`page_icon`**: Uses the 🤖 robot emoji as the icon displayed in the browser tab.
     - **`layout`**: Sets the app layout to "wide" for a more expansive view.
     - **`initial_sidebar_state`**: Ensures the sidebar is **expanded** when the app is first loaded.

2. **Navigation Sidebar**:
   - The `navigation_sidebar()` function is called to render the sidebar, which provides navigation buttons for the different pages in the app. It ensures the app is ready for seamless navigation.

3. **Page Routing**:
   - If `st.session_state.get('current_page')` exists (i.e., a page has been selected), the app dynamically loads the corresponding page from the `PAGES` dictionary.
   - The `PAGES` dictionary maps page names (e.g., "Landing", "Resume Analyzer", "Interview Coach", "Report") to their respective functions.
   - If no page is selected (on the first visit or if the session state is empty), the **Landing Page** is loaded by default.

4. **Page Rendering**:
   - Based on the current page stored in `st.session_state.current_page`, the corresponding page function (e.g., `landing_page`, `resume_analyzer`, `interview_coach`, or `report_dashboard`) is called to display the content for that page.
   
5. **Default Landing Page**:
   - If no specific page is selected, the `landing_page()` function is called to display the default landing page of the app.

---

#### 🎨 **Key Features**:
- **Dynamic Page Rendering**: Based on the current page stored in the session state, the app renders the corresponding page dynamically.
- **Seamless User Experience**: The app automatically loads the last visited page or defaults to the landing page, ensuring a smooth experience when navigating through sections.
- **Responsive Sidebar**: The sidebar is integrated at the top level, so users can switch between pages without losing context.

---

#### 🚀 **How to Use**:
1. **App Initialization**: On launching the app, the `main()` function is called, configuring the app's settings, rendering the navigation sidebar, and loading the appropriate page.
2. **Page Navigation**: After the user clicks a navigation button (via the sidebar), the app dynamically updates to show the selected page (Resume Analyzer, Interview Coach, or Report).
3. **Automatic Page Loading**: When the app is first loaded, the **Landing Page** is displayed. After that, the app remembers the last selected page and displays it on subsequent visits.

---

The **Main App Flow** provides the foundation for the **IntelliView** app's functionality, ensuring a smooth, user-friendly experience by dynamically routing between pages and maintaining session state across interactions.

In [ ]:
# Cell 12: Main App Flow
%%writefile app.py -a
def main():
    st.set_page_config(
        page_title="IntelliView",
        page_icon="🤖",
        layout="wide",
        initial_sidebar_state="expanded"
    )
    navigation_sidebar()
    if st.session_state.get('current_page'):
        PAGES = {
            "Landing": landing_page,
            "Resume Analyzer": resume_analyzer,
            "Interview Coach": interview_coach,
            "Report": report_dashboard
        }
        PAGES[st.session_state.current_page]()
    else:
        landing_page()

if __name__ == "__main__":
    main()

---

# **Exposing the Streamlit App with Ngrok**

In this section, we use **ngrok** to expose the locally running **Streamlit** app to the internet. This allows you to access the app from any browser using a public URL, which is especially useful for sharing your app or testing it remotely. Here's a breakdown of the code:

1. **Imports**:
    - `os`: This is used to run system commands. We use it to execute the command that starts the Streamlit app.
    - `Thread` from `threading`: This allows us to run the Streamlit app in the background without blocking the rest of the program.
    - `ngrok` from `pyngrok`: This creates a secure tunnel from the local machine to the internet, allowing external access to the app.

2. **Running the Streamlit App**:
    - The function `run_streamlit()` starts the Streamlit app by running the command: `streamlit run app.py --server.port 8500`. This starts the app on port `8500`.

3. **Creating a Background Thread**:
    - A `Thread` object is created to run the Streamlit app in the background. This allows the script to continue executing without waiting for Streamlit to finish.

4. **Setting Up Ngrok**:
    - `public_url = ngrok.connect(addr='8500', proto='http', bind_tls=True)`: This line creates a secure (HTTPS) ngrok tunnel to expose the Streamlit app running on port `8500`. The `bind_tls=True` ensures that the connection is secure.
    
5. **Displaying the Public URL**:
    - Once the tunnel is created, the public URL is printed using `print('App is live at', public_url)`. This URL is the one you'll use to access the Streamlit app from anywhere.

In [ ]:
import os
from threading import Thread
from pyngrok import ngrok
def run_streamlit():
    os.system('streamlit run app.py --server.port 8500')
thread = Thread(target=run_streamlit)
thread.start()
public_url = ngrok.connect(addr='8500', proto='http', bind_tls=True)
print('App is live at', public_url)